In [44]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import homogeneity_score, adjusted_rand_score, adjusted_mutual_info_score
from time import time
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [45]:
df = pd.read_csv('./final.csv')

In [46]:
df

customer_unique_id  order_item_id  review_score  \
0      0000366f3b9a7992bf8c76cfdf3221e2            1.0           5.0   
1      0000b849f77a49e4a4ce2b2a4ca5be3f            1.0           4.0   
2      0000f46a3911fa3c0805444483337064            1.0           3.0   
3      0000f6ccb0745a6a4b88665a16c9f078            1.0           4.0   
4      0004aac84e0df4da2b147fca70cf8255            1.0           5.0   
...                                 ...            ...           ...   
91472  fffcf5a5ff07b0908bd4e2dbc735a684            3.0           5.0   
91473  fffea47cd6d3cc0a88bd621562a9d061            1.0           4.0   
91474  ffff371b4d645b6ecea244b27531430a            1.0           5.0   
91475  ffff5962728ec6157033ef9805bacc48            1.0           5.0   
91476  ffffd2657e2aad2907e67c3e9daecbeb            1.0           5.0   

       payment_value  payment_sequential  freight_value   price  label_kmeans  
0             141.90                 1.0          12.00  129.90             2  
1              27.19                 1.0           8.29   18.90             2  
2              86.22                 1.0          17.22   69.00             2  
3              43.62                 1.0          17.63   25.99             2  
4             196.89                 1.0          16.89  180.00             2  
...              ...                 ...            ...     ...           ...  
91472        4134.84                 1.0         258.28  890.00             3  
91473          84.58                 1.0          19.69   64.89             2  
91474         112.46                 1.0          22.56   89.90             2  
91475         133.69                 1.0          18.69  115.00             2  
91476          71.56                 1.0          14.57   56.99             2  

[91477 rows x 8 columns]

In [47]:
df_agg = pd.read_csv('./agg.csv')

In [48]:
df_agg

customer_unique_id  order_item_id  review_score  \
0      0000366f3b9a7992bf8c76cfdf3221e2            1.0           5.0   
1      0000b849f77a49e4a4ce2b2a4ca5be3f            1.0           4.0   
2      0000f46a3911fa3c0805444483337064            1.0           3.0   
3      0000f6ccb0745a6a4b88665a16c9f078            1.0           4.0   
4      0004aac84e0df4da2b147fca70cf8255            1.0           5.0   
...                                 ...            ...           ...   
91472  fffcf5a5ff07b0908bd4e2dbc735a684            3.0           5.0   
91473  fffea47cd6d3cc0a88bd621562a9d061            1.0           4.0   
91474  ffff371b4d645b6ecea244b27531430a            1.0           5.0   
91475  ffff5962728ec6157033ef9805bacc48            1.0           5.0   
91476  ffffd2657e2aad2907e67c3e9daecbeb            1.0           5.0   

       payment_value  payment_sequential payment_type  freight_value   price  \
0             141.90                 1.0  credit_card          12.00  129.90   
1              27.19                 1.0  credit_card           8.29   18.90   
2              86.22                 1.0  credit_card          17.22   69.00   
3              43.62                 1.0  credit_card          17.63   25.99   
4             196.89                 1.0  credit_card          16.89  180.00   
...              ...                 ...          ...            ...     ...   
91472        4134.84                 1.0  credit_card         258.28  890.00   
91473          84.58                 1.0  credit_card          19.69   64.89   
91474         112.46                 1.0  credit_card          22.56   89.90   
91475         133.69                 1.0  credit_card          18.69  115.00   
91476          71.56                 1.0  credit_card          14.57   56.99   

                   customer_unique_id.1 order_purchase_timestamp  
0      0000366f3b9a7992bf8c76cfdf3221e2      2018-05-10 10:56:27  
1      0000b849f77a49e4a4ce2b2a4ca5be3f      2018-05-07 11:11:27  
2      0000f46a3911fa3c0805444483337064      2017-03-10 21:05:03  
3      0000f6ccb0745a6a4b88665a16c9f078      2017-10-12 20:29:41  
4      0004aac84e0df4da2b147fca70cf8255      2017-11-14 19:45:42  
...                                 ...                      ...  
91472  fffcf5a5ff07b0908bd4e2dbc735a684      2017-06-08 21:00:36  
91473  fffea47cd6d3cc0a88bd621562a9d061      2017-12-10 20:07:56  
91474  ffff371b4d645b6ecea244b27531430a      2017-02-07 15:49:16  
91475  ffff5962728ec6157033ef9805bacc48      2018-05-02 15:17:41  
91476  ffffd2657e2aad2907e67c3e9daecbeb      2017-05-02 20:18:45  

[91477 rows x 10 columns]

In [49]:
df['date_last_order'] = df_agg['order_purchase_timestamp']

In [50]:
df

customer_unique_id  order_item_id  review_score  \
0      0000366f3b9a7992bf8c76cfdf3221e2            1.0           5.0   
1      0000b849f77a49e4a4ce2b2a4ca5be3f            1.0           4.0   
2      0000f46a3911fa3c0805444483337064            1.0           3.0   
3      0000f6ccb0745a6a4b88665a16c9f078            1.0           4.0   
4      0004aac84e0df4da2b147fca70cf8255            1.0           5.0   
...                                 ...            ...           ...   
91472  fffcf5a5ff07b0908bd4e2dbc735a684            3.0           5.0   
91473  fffea47cd6d3cc0a88bd621562a9d061            1.0           4.0   
91474  ffff371b4d645b6ecea244b27531430a            1.0           5.0   
91475  ffff5962728ec6157033ef9805bacc48            1.0           5.0   
91476  ffffd2657e2aad2907e67c3e9daecbeb            1.0           5.0   

       payment_value  payment_sequential  freight_value   price  label_kmeans  \
0             141.90                 1.0          12.00  129.90             2   
1              27.19                 1.0           8.29   18.90             2   
2              86.22                 1.0          17.22   69.00             2   
3              43.62                 1.0          17.63   25.99             2   
4             196.89                 1.0          16.89  180.00             2   
...              ...                 ...            ...     ...           ...   
91472        4134.84                 1.0         258.28  890.00             3   
91473          84.58                 1.0          19.69   64.89             2   
91474         112.46                 1.0          22.56   89.90             2   
91475         133.69                 1.0          18.69  115.00             2   
91476          71.56                 1.0          14.57   56.99             2   

           date_last_order  
0      2018-05-10 10:56:27  
1      2018-05-07 11:11:27  
2      2017-03-10 21:05:03  
3      2017-10-12 20:29:41  
4      2017-11-14 19:45:42  
...                    ...  
91472  2017-06-08 21:00:36  
91473  2017-12-10 20:07:56  
91474  2017-02-07 15:49:16  
91475  2018-05-02 15:17:41  
91476  2017-05-02 20:18:45  

[91477 rows x 9 columns]

In [51]:
df = df.drop(['customer_unique_id'], axis=1)

In [52]:
df.dtypes

order_item_id         float64
review_score          float64
payment_value         float64
payment_sequential    float64
freight_value         float64
price                 float64
label_kmeans            int64
date_last_order        object
dtype: object

In [53]:
df_ari = df[df['date_last_order'].str.contains('2016')]

In [54]:
df_ari

order_item_id  review_score  payment_value  payment_sequential  \
72               1.0           4.0         144.94                 1.0   
711              1.0           5.0         784.80                 1.0   
2220             1.0           4.0         971.96                 1.0   
2409             1.0           5.0         196.27                 1.0   
2963             1.0           3.0         267.49                 1.0   
...              ...           ...            ...                 ...   
89822            6.0           1.0         544.77                 1.0   
90636            1.0           5.0        1423.55                 1.0   
91270            3.0           5.0        1069.96                 1.0   
91311            1.0           4.0          98.83                 1.0   
91383            3.0           5.0         218.04                 1.0   

       freight_value    price  label_kmeans      date_last_order  
72             16.04   128.90             2  2016-10-06 11:32:00  
711            44.82   739.98             3  2016-10-10 13:57:41  
2220           21.97   949.99             3  2016-10-08 02:18:43  
2409           16.37   179.90             2  2016-10-04 23:39:01  
2963           17.59   249.90             2  2016-10-04 13:15:46  
...              ...      ...           ...                  ...  
89822          15.63    44.90             2  2016-10-07 00:54:40  
90636          24.55  1399.00             3  2016-10-08 01:28:14  
91270          17.59   249.90             3  2016-10-06 10:47:15  
91311          18.84    79.99             2  2016-10-09 22:38:35  
91383          12.41    47.40             2  2016-10-09 23:25:13  

[248 rows x 8 columns]

In [55]:
df_ari = df_ari.drop('date_last_order',axis = 1)

In [56]:
myList2 = [] # creating an empty list
for i in range(9):
    myList2.append(1 + i)

In [57]:
myList2

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [58]:
for i in myList2:
    if i == 1:
        df_ari_1 = df[df['date_last_order'].str.contains('2017-0' + str(i))]
        df_ari_1 = df_ari_1.drop('date_last_order', axis = 1)
        df_ari_2017_6_mois = df_ari.append(df_ari_1, ignore_index=True)
    else:
        df_ari_1 = df[df['date_last_order'].str.contains('2017-0' + str(i))]
        df_ari_1 = df_ari_1.drop('date_last_order', axis = 1)
        df_ari_2017_6_mois = df_ari_2017_6_mois.append(df_ari_1, ignore_index=True)

In [59]:
df_ari_2017_6_mois

order_item_id  review_score  payment_value  payment_sequential  \
0                1.0           4.0         144.94                 1.0   
1                1.0           5.0         784.80                 1.0   
2                1.0           4.0         971.96                 1.0   
3                1.0           5.0         196.27                 1.0   
4                1.0           3.0         267.49                 1.0   
...              ...           ...            ...                 ...   
24428            1.0           4.0          26.68                 1.0   
24429            1.0           4.0         104.37                 1.0   
24430            1.0           5.0         905.76                 1.0   
24431            1.0           4.0          62.94                 1.0   
24432            1.0           4.0          55.00                 1.0   

       freight_value   price  label_kmeans  
0              16.04  128.90             2  
1              44.82  739.98             3  
2              21.97  949.99             3  
3              16.37  179.90             2  
4              17.59  249.90             2  
...              ...     ...           ...  
24428           7.78   18.90             2  
24429          14.38   89.99             2  
24430          18.76  887.00             3  
24431           7.94   55.00             2  
24432          15.10   39.90             2  

[24433 rows x 7 columns]

In [60]:
df_ari_1

order_item_id  review_score  payment_value  payment_sequential  \
15               2.0           4.5          93.70                 1.0   
142              1.0           5.0          49.60                 1.0   
147              1.0           5.0         236.81                 1.0   
148              1.0           5.0         204.83                 1.0   
158              1.0           4.0          94.35                 1.0   
...              ...           ...            ...                 ...   
91377            1.0           4.0          26.68                 1.0   
91416            1.0           4.0         104.37                 1.0   
91432            1.0           5.0         905.76                 1.0   
91458            1.0           4.0          62.94                 1.0   
91467            1.0           4.0          55.00                 1.0   

       freight_value   price  label_kmeans  
15             11.85   35.00             2  
142            17.60   32.00             2  
147            36.91  199.90             2  
148            14.93  189.90             2  
158             9.41   84.94             2  
...              ...     ...           ...  
91377           7.78   18.90             2  
91416          14.38   89.99             2  
91432          18.76  887.00             3  
91458           7.94   55.00             2  
91467          15.10   39.90             2  

[3865 rows x 7 columns]

In [61]:
kmeans_ari_1 = KMeans(n_clusters=4, random_state=7).fit(df_ari_2017_6_mois)
silhouette_score(df_ari_2017_6_mois,kmeans_ari_1.labels_)

0.827358180337376

In [62]:
df_result = pd.DataFrame(kmeans_ari_1.labels_, columns = ['label_ari_2017'])

In [63]:
df_result

label_ari_2017
0                   0
1                   3
2                   3
3                   0
4                   0
...               ...
24428               0
24429               0
24430               3
24431               0
24432               0

[24433 rows x 1 columns]

In [64]:
myList3 = [] # creating an empty list
for i in range(3):
    myList3.append(10 + i)

In [65]:
myList3

[10, 11, 12]

In [66]:
myList = [] # creating an empty list
for i in range(8):
    myList.append(1 + i)

In [67]:
myList

[1, 2, 3, 4, 5, 6, 7, 8]

In [68]:
maboucle = []
for i in range(11):
    maboucle.append(1 + i)

In [69]:
maboucle

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [70]:
annee = 2017
for i in maboucle :
    if i == 1:
        a = 10
    if a == 10 :
        df_ari_2 = df[df['date_last_order'].str.contains('2017-' + str(a))]
        df_ari_2 = df_ari_2.drop('date_last_order', axis = 1)
        df_ari_tot = df_ari_2017_6_mois.append(df_ari_2, ignore_index=True)
        kmeans_ari = KMeans(n_clusters=4, random_state=7).fit(df_ari_tot)
        df_result_2 = pd.DataFrame(kmeans_ari.labels_, columns = ['label_ari_2017_' + str(a)])
        df_result_total1 = pd.merge(df_result, df_result_2, left_index=True, right_index=True, how="outer")
        df_result_total1 = df_result_total1.dropna()        
        print('Il y a ' + str(df_ari_tot.shape[0] - df_result.shape[0]) + ' lignes ajoutés')
        print('Le mois numéro ' + str(a) + ' de 2017 le score ARI est de ' + str(adjusted_rand_score(df_result_total1['label_ari_2017'], df_result_total1['label_ari_2017_' + str(a)])))
        a = a + 1
    elif a == 12:
        mylist3 = myList
        df_ari_2 = df[df['date_last_order'].str.contains('2017-' + str(a))]
        df_ari_2 = df_ari_2.drop('date_last_order', axis = 1)
        df_ari_tot = df_ari_tot.append(df_ari_2, ignore_index=True)
        kmeans_ari = KMeans(n_clusters=4, random_state=7).fit(df_ari_tot)
        df_result_2 = pd.DataFrame(kmeans_ari.labels_, columns = ['label_ari_2017_' + str(a)])
        df_result_total1 = pd.merge(df_result_total1, df_result_2, left_index=True, right_index=True, how="outer")
        df_result_total1 = df_result_total1.dropna()
        print('Il y a ' + str(df_ari_tot.shape[0] - df_result.shape[0]) + ' lignes ajoutés')
        print('Le mois numéro ' + str(a) + ' de 2017 le score ARI est de ' + str(adjusted_rand_score(df_result_total1['label_ari_2017'], df_result_total1['label_ari_2017_' + str(a)])))
        annee = annee + 1
        a = 1
    elif annee == 2018 :
        df_ari_2 = df[df['date_last_order'].str.contains('2018-0' + str(a))]
        df_ari_2 = df_ari_2.drop('date_last_order', axis = 1)
        df_ari_tot = df_ari_tot.append(df_ari_2, ignore_index=True)
        kmeans_ari = KMeans(n_clusters=4, random_state=7).fit(df_ari_tot)
        df_result_2 = pd.DataFrame(kmeans_ari.labels_, columns = ['label_ari_2018_0' + str(a)])
        df_result_total1 = pd.merge(df_result_total1, df_result_2, left_index=True, right_index=True, how="outer")
        df_result_total1 = df_result_total1.dropna()
        print('Il y a ' + str(df_ari_tot.shape[0] - df_result.shape[0]) + ' lignes ajoutés')
        print('Le mois numéro ' + str(a) + ' de 2018 le score ARI est de ' + str(adjusted_rand_score(df_result_total1['label_ari_2017'], df_result_total1['label_ari_2018_0' + str(a)])))
        a = a + 1
    else:
        df_ari_2 = df[df['date_last_order'].str.contains('2017-' + str(a))]
        df_ari_2 = df_ari_2.drop('date_last_order', axis = 1)
        df_ari_tot = df_ari_tot.append(df_ari_2, ignore_index=True)
        kmeans_ari = KMeans(n_clusters=4, random_state=7).fit(df_ari_tot)
        df_result_2 = pd.DataFrame(kmeans_ari.labels_, columns = ['label_ari_2017_' + str(a)])
        df_result_total1 = pd.merge(df_result_total1, df_result_2, left_index=True, right_index=True, how="outer")
        df_result_total1 = df_result_total1.dropna()
        print('Il y a ' + str(df_ari_tot.shape[0] - df_result.shape[0]) + ' lignes ajoutés')
        print('Le mois numéro ' + str(a) + ' de 2017 le score ARI est de ' + str(adjusted_rand_score(df_result_total1['label_ari_2017'], df_result_total1['label_ari_2017_' + str(a)])))
        a = a + 1

Il y a 4222 lignes ajoutés
Le mois numéro 10 de 2017 le score ARI est de 0.9752445757769606
Il y a 11111 lignes ajoutés
Le mois numéro 11 de 2017 le score ARI est de 0.550709884478501
Il y a 16299 lignes ajoutés
Le mois numéro 12 de 2017 le score ARI est de 0.5574151914902157
Il y a 22921 lignes ajoutés
Le mois numéro 1 de 2018 le score ARI est de 0.7715328112448598
Il y a 29064 lignes ajoutés
Le mois numéro 2 de 2018 le score ARI est de 0.5728644614265295
Il y a 35723 lignes ajoutés
Le mois numéro 3 de 2018 le score ARI est de 0.7761030575663727
Il y a 42262 lignes ajoutés
Le mois numéro 4 de 2018 le score ARI est de 0.5772289009540886
Il y a 48827 lignes ajoutés
Le mois numéro 5 de 2018 le score ARI est de 0.5514574486356729
Il y a 54797 lignes ajoutés
Le mois numéro 6 de 2018 le score ARI est de 0.5514574486356729
Il y a 60783 lignes ajoutés
Le mois numéro 7 de 2018 le score ARI est de 0.5514574486356729
Il y a 67044 lignes ajoutés
Le mois numéro 8 de 2018 le score ARI est de 0.5514

In [71]:
df_result_total1

label_ari_2017  label_ari_2017_10  label_ari_2017_11  \
0                 0.0                0.0                0.0   
1                 3.0                2.0                0.0   
2                 3.0                2.0                3.0   
3                 0.0                0.0                0.0   
4                 0.0                0.0                0.0   
...               ...                ...                ...   
24428             0.0                0.0                0.0   
24429             0.0                0.0                0.0   
24430             3.0                2.0                0.0   
24431             0.0                0.0                0.0   
24432             0.0                0.0                0.0   

       label_ari_2017_12  label_ari_2018_01  label_ari_2018_02  \
0                    0.0                0.0                3.0   
1                    0.0                2.0                3.0   
2                    3.0                2.0                0.0   
3                    0.0                0.0                3.0   
4                    0.0                0.0                3.0   
...                  ...                ...                ...   
24428                0.0                0.0                3.0   
24429                0.0                0.0                3.0   
24430                0.0                2.0                0.0   
24431                0.0                0.0                3.0   
24432                0.0                0.0                3.0   

       label_ari_2018_03  label_ari_2018_04  label_ari_2018_05  \
0                    0.0                0.0                0.0   
1                    3.0                0.0                0.0   
2                    3.0                3.0                3.0   
3                    0.0                0.0                0.0   
4                    0.0                0.0                0.0   
...                  ...                ...                ...   
24428                0.0                0.0                0.0   
24429                0.0                0.0                0.0   
24430                3.0                3.0                0.0   
24431                0.0                0.0                0.0   
24432                0.0                0.0                0.0   

       label_ari_2018_06  label_ari_2018_07  label_ari_2018_08  
0                    0.0                0.0                  0  
1                    0.0                0.0                  0  
2                    2.0                2.0                  1  
3                    0.0                0.0                  0  
4                    0.0                0.0                  0  
...                  ...                ...                ...  
24428                0.0                0.0                  0  
24429                0.0                0.0                  0  
24430                0.0                0.0                  0  
24431                0.0                0.0                  0  
24432                0.0                0.0                  0  

[24433 rows x 12 columns]